In [1]:
import os
import sys

up1 = os.path.abspath('..') 
sys.path.insert(0, up1)
import sys
print(sys.path)

['/data/su_buildup/su_buildup', '', '/home/ubuntu/src/cntk/bindings/python', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python36.zip', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/lib-dynload', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages', '/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/extensions', '/home/ubuntu/.ipython']


In [2]:

import random
import cv2, numpy as np
import pickle


from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, RepeatVector
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.applications.vgg16 import VGG16
from keras import regularizers
from keras.callbacks import History 
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model

from keras_data_generator.ucf101_datagenerator.generator_class import DataGenerator
import csv



/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


## The VGG 16 model of spatial training

    So the idea here is that we're building a VGG 16 model without the top included. Essentially we want all the feature
    learning component of the model without the fully connected layer and input layer. We will also freeze VGG 16 FL layers 
    as we simply want it to learn how to classify based on the input from the sport action recognition layer. 

In [3]:
# Parameters
params = { 'data_dir' : "/data",
          'dim': (224,224),
          'batch_size': 64,
          'n_classes': 101,
          'n_channels': 3,
          'shuffle': True}


In [4]:
def getVggModel(LR,l1v,drp,printmod=1):

    base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
    x = base_model.get_layer('block5_pool').output   # collect outputs from hidden layer, Block 5
    # stitch layers to the VGG16 layers
    
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)

    predictions = Dense(101, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # freeze original VGG16 layers
    for i, layer in enumerate(model.layers):
        if 'block1' in layer.name or 'block2' in layer.name or 'block3' in layer.name or \
        'block4' in layer.name or 'block5' in layer.name:
            layer.trainable = False
        else:
            layer.trainable = True
            
    mypotim = SGD(lr=LR, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    if (printmod==1 ):
        model.summary()
    return model

In [5]:

def getData(listPath):
# load Id list and labels 

    train_list = list()

    with open(listPath, 'r') as f:
        reader = csv.reader(f)
        train_list = list(reader)

        labels = [int(label[0].split(' ')[1]) for label in train_list ]
    IDs = [label[0].split(' ')[0] for label in train_list ]
    # IDs
    IDs = [id.split('/')[1].rstrip('.avi') for id in IDs ]

    labels = dict(zip(IDs, labels))
    return (IDs, labels)

id_labels_train = getData('../keras_data_generator/ucf101_splits/trainlist01.txt')
id_labels_test = getData('../keras_data_generator/ucf101_splits/trainlist02.txt')

training_generator = DataGenerator(*id_labels_train, **params)
validation_generator = DataGenerator(*id_labels_test, **params)


In [4]:
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

def get_callbacks(filepath, patience=10):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

In [5]:
model=getVggModel(1e-4,1e-1,0.9)



58892288/58889256 [==============================] - 4s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool

In [8]:
# Train model on dataset
mod1 = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=2, epochs=10,
                    verbose=1)

NameError: name 'training_generator' is not defined

In [3]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

# A better Image  Generator

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        '/data/sym_datasets/jpegs_256_1_train/',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',
        follow_links=True
        )

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        '/data/sym_datasets/jpegs_256_1_test/',
        target_size=(224, 224),
        batch_size=64,
        class_mode='categorical',
        follow_links=True)


Found 10243 images belonging to 101 classes.
Found 10242 images belonging to 101 classes.


In [8]:
# Train model on dataset
mod1 = model.fit_generator(generator=train_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=2, epochs=10,
                    verbose=1)

Epoch 1/10
161/161 [==============================] - 240s 1s/step - loss: 4.5776 - acc: 0.0307 - val_loss: 4.4894 - val_acc: 0.0319
Epoch 2/10
161/161 [==============================] - 184s 1s/step - loss: 4.4401 - acc: 0.0520 - val_loss: 4.3637 - val_acc: 0.0637
Epoch 3/10
161/161 [==============================] - 184s 1s/step - loss: 4.3238 - acc: 0.0714 - val_loss: 4.2391 - val_acc: 0.0763
Epoch 4/10
161/161 [==============================] - 184s 1s/step - loss: 4.2032 - acc: 0.0890 - val_loss: 4.1304 - val_acc: 0.1109
Epoch 5/10
161/161 [==============================] - 184s 1s/step - loss: 4.0904 - acc: 0.1131 - val_loss: 3.9908 - val_acc: 0.1261
Epoch 6/10
161/161 [==============================] - 184s 1s/step - loss: 3.9752 - acc: 0.1428 - val_loss: 3.8817 - val_acc: 0.1545
Epoch 7/10
161/161 [==============================] - 184s 1s/step - loss: 3.8563 - acc: 0.1652 - val_loss: 3.7612 - val_acc: 0.2160
Epoch 8/10
161/161 [==============================] - 184s 1s/step - 

In [9]:
mod1 = model.fit_generator(generator=train_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=2, epochs=20,
                    verbose=1)

Epoch 1/20
161/161 [==============================] - 186s 1s/step - loss: 3.4358 - acc: 0.2681 - val_loss: 3.3430 - val_acc: 0.2872
Epoch 2/20
161/161 [==============================] - 184s 1s/step - loss: 3.3536 - acc: 0.2906 - val_loss: 3.2519 - val_acc: 0.2885
Epoch 3/20
161/161 [==============================] - 183s 1s/step - loss: 3.2703 - acc: 0.3021 - val_loss: 3.1602 - val_acc: 0.3187
Epoch 4/20
161/161 [==============================] - 184s 1s/step - loss: 3.1852 - acc: 0.3219 - val_loss: 3.0715 - val_acc: 0.3389
Epoch 5/20
161/161 [==============================] - 184s 1s/step - loss: 3.0936 - acc: 0.3433 - val_loss: 2.9841 - val_acc: 0.3588
Epoch 6/20
161/161 [==============================] - 181s 1s/step - loss: 3.0171 - acc: 0.3531 - val_loss: 2.9043 - val_acc: 0.3953
Epoch 7/20
161/161 [==============================] - 181s 1s/step - loss: 2.9391 - acc: 0.3733 - val_loss: 2.8273 - val_acc: 0.4068
Epoch 8/20
161/161 [==============================] - 181s 1s/step - 

In [12]:
mod1['history']

TypeError: 'History' object is not subscriptable